# Exploratory Data Analysis

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Import the dataset as a CSV
df = pd.read_csv('0_churn_dataset.csv')
df.head()

In [ ]:
# Show summary statistics for all columns
df.describe()

In [ ]:
# check for null values
df.isnull().sum()
# no null values, so no need to drop any rows

In [ ]:
# check for duplicate rows
duplicates = df.duplicated().sum()
print(f"Number of Duplicate Rows:{duplicates}")
# no duplicate rows, so no need to drop any rows

In [ ]:
#Check variables and associated data type
df.dtypes
# Looks fine as well

In [ ]:
# Insight 1: Finding out proportion of customers who have churned in total
total_churned = df['Churn'].sum()
total_customers = df.shape[0]
proportion = (total_churned / total_customers) * 100
print(f"Insight 1: {proportion}% of customers have churned in total")

This means the dataset is unbalanced and we need to be careful building the model so it can predict both churners and non-churners well

In [ ]:
# Insight 2: Finding out if churned customers have higher customer service calls than non-churned customers
non_churned = df[df['Churn'] == False]
churned = df[df['Churn'] == True]

avg_service_calls_churned = churned['Customer service calls'].mean()
avg_service_calls_non_churned = non_churned['Customer service calls'].mean()

print(f"Average customer service calls for churned customers: {avg_service_calls_churned}")
print(f"Average customer service calls for non-churned customers: {avg_service_calls_non_churned}")

if avg_service_calls_churned > avg_service_calls_non_churned:
    print("Insight 2: Churned customers have higher average nr. of customer service calls than non-churned customers.")
else:
    print("Insight 2: Churned customers do not have higher average nr. of customer service calls than non-churned customers.")


In [ ]:
#Insight 3: Find out if customers who were charged more than the average monthly charge are more likely to churn
# Adding column that sums up total day charge, total eve charge, total night charge and total intl charge
insight_3_df = df.copy()
insight_3_df['Total charge'] = insight_3_df['Total day charge'] + insight_3_df['Total eve charge'] + insight_3_df['Total night charge'] + insight_3_df['Total intl charge']
insight_3_df["Total charge"].describe()

In [ ]:
# Calculating likelihood of churn for customers who were charged more than the average monthly charge
# Calculate average charge
avg_charge = insight_3_df['Total charge'].mean()
# Extracting churned customers who were charged more than the average monthly charge
churned_above_avg_charge = insight_3_df[insight_3_df['Total charge'] > avg_charge]
churned_above_avg_charge.head()

In [ ]:
# Calculate churn rate for customers who were charged more than the average monthly charge
churn_rate_above_avg_charge = churned_above_avg_charge['Churn'].mean() * 100
print(f"The churn rate for customers who were charged more than the average monthly charge is {churn_rate_above_avg_charge}%")
#Calculate average churn rate for all customers
avg_churn_rate = insight_3_df['Churn'].mean() * 100
print(f"The average churn rate for all customers is {avg_churn_rate}%")
print(f"Insight 3: Customers who were charged more than the average monthly charge are more likely to churn than the average customer")

# End-to-end machine learning pipeline

This is a classification problem because the target variable/label "Churn" is a binary variable that takes on one of two possible values, namely "True" or "False". Following, two different machine learning models will be trained to achieve as high of a F-2 score as possible.

The F-2 score is chosen because it merges recall and precision into a single score, providing a good blend between how well the model can predict churners well and if it overpredicts non-churners to be churners. Unlike the F-1 score, the F-2 score puts more emphasis on recall than on precision. This makes sense because the telcom company should be more concerned about correctly predicting all possible churning customers (recall) than predicting too many non-churning customers to be churners (precision) because a churned customer likely costs the company a lot more than being overcareful with a non-churning customer. On the other hand, just focusing entirely on recall will incentivize the model just to predict all customers as churners which is not ideal because it will involve a lot of unnecessary effort on the part of the telcom company to keep customers happy that are not planning on leaving in the first place. Therefore, the F-2 score provides a very good balance and makes most sense to me as a performance metric.

#### Model 1: Logistic Regression

This model is chosen because Logistic regression is a simple classification algorithm that can predict the probability of a binary response belonging to one class or the other. This is exactly the kind of prediction problem we have for our dataset where we predict if a customer churns or not.

##### Step 1: Feature Selection

In [ ]:
# Model 1: Logistic Regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Feature Selection
# Drop state column as it is a categorical variable with too many options to be included in the model
model1_df = df.drop('State', axis=1)
# Make categorical columns International Plan and Voice Mail Plan into binary columns with 1 for Yes and 0 for No, do the same for Churn
model1_df['International plan'] = model1_df['International plan'].map({'Yes': 1, 'No': 0})
model1_df['Voice mail plan'] = model1_df['Voice mail plan'].map({'Yes': 1, 'No': 0})
model1_df['Churn'] = model1_df['Churn'].astype(int)
model1_df.head()

In [ ]:
# Look at the correlation matrix to see which variables are correlated with Churn and should be included in the model
plt.figure(figsize=(16, 12))
sns.heatmap(model1_df.corr(), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# As total eve calls is perfectly correlated with total eve charge and total eve minutes, we will drop total eve calls and total eve minutes to avoid multicollinearity
model1_df = model1_df.drop(['Total eve calls', 'Total eve minutes'], axis=1)
# Do the same for total day calls and total day minutes, total night calls and total night minutes as well as total intl calls and total intl minutes
model1_df = model1_df.drop(['Total day calls', 'Total day minutes', 'Total night calls', 'Total night minutes', 'Total intl calls', 'Total intl minutes'], axis=1)
# Also drop area code column as it barely correlates with churn and should also be categorical, not numerical
model1_df = model1_df.drop('Area code', axis=1)
model1_df.head()

In [ ]:
# As Number vmail messages is very highly correlated with Voice mail plan_Yes and Number vmail messages contains
# more information, we will drop Voice mail plan_Yes to avoid multicollinearity
model1_df = model1_df.drop(['Voice mail plan'], axis=1)
model1_df.head()

In [ ]:
# Creating a descriptive table with min, median, mean, max values for the numerical columns
model1_df.describe()

##### Step 2: Train-test split

In [ ]:
# Splitting the data into training and testing sets (80/20 split)
X = model1_df.drop(['Churn'], axis=1)  # Features
y = model1_df['Churn']  # Target variable

# Using stratify=y to ensure that the train and test sets have approximately the same proportion of churn instances
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Check the shapes of the train and test sets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Check the distribution of the target variable in train and test sets to ensure that stratify=y worked
print(f"Churn rate in training set: {y_train.mean() * 100:.2f}%")
print(f"Churn rate in test set: {y_test.mean() * 100:.2f}%")

##### Step 3: Hyperparameter tuning

In [ ]:
# For logistic regression, a suitable hyperparameter is the lambda parameter for regularization to prevent overfitting

# Use of a simple grid search, namely ElasticNetCV, to find the optimal lambda parameter
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import StratifiedKFold

# Create a stratified cross-validation object to split the data in a way
# that preserves the percentage of samples for each class
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Alphas and l1_ratio parameters to search
# alpha is the lambda parameter for regularization controlling the strength of the penalty for including more features in the model
# l1_ratio is the ratio of L1 regularization to L2 regularization
alphas = [0.0001, 0.001, .01, .1, 1, 10, 100]
l1_ratios = [0.0001, .001, .01, .1, .2, .4, .8, .99, .999, 1]

# Create ElasticNetCV object with stratified cross-validation
elastic_net = ElasticNetCV(
    l1_ratio=l1_ratios,
    alphas=alphas,
    cv=cv,
    random_state=42,
    selection='random'  # For faster computation
)
# Fit the model to the training data
elastic_net.fit(X_train, y_train)

# Print the optimal l1_ratio parameter
print(f"Optimal l1_ratio parameter: {elastic_net.l1_ratio_}")

# if logic for the optimal l1_ratio parameter, if lower than 0.5, more L2 regularization, if higher than 0.5, more L1 regularization
if elastic_net.l1_ratio_ < 0.5:
    print(f"A l1_ratio of {elastic_net.l1_ratio_} means that the optimal model uses more L2 regularization than L1 regularization, so it tries to set some coefficients to zero.")
else:
    print(f"""A l1_ratio of {elastic_net.l1_ratio_} means that the optimal model uses more L1 regularization than L2 regularization, 
          so it tries to reduce the coefficients as much as possible but does not set them to zero.""")

# Print the optimal lambda parameter
print(f"\nOptimal lambda parameter: {elastic_net.alpha_}")

# if logic for the optimal lambda parameter, if lower than 0.1, low regularization, if between 0.1 and 1, moderate regularization, if higher than 1, high regularization
if elastic_net.alpha_ < 0.1:
    print(f"A lambda parameter of {elastic_net.alpha_} means that the optimal model only penalizes the coefficients slightly, so it does not use much regularization.")
elif 0.1 <= elastic_net.alpha_ <= 1:
    print(f"A lambda parameter of {elastic_net.alpha_} means that the optimal model uses moderate regularization.")
else:
    print(f"A lambda parameter of {elastic_net.alpha_} means that the optimal model uses high regularization.")

The low lambda parameter makes sense as we have already removed highly correlated variables and have a small number of features.

One further parameter to change is the decision threshhold, however, this is not a hyperparameter in the strictest  sense as it does not impact the model's training process. We will use the default decision threshold of 0.5 and try different thresholds later on.

##### Step 4: Training the Model

In [ ]:
# Training the logistic regression model on the test-split of the data
log_reg = LogisticRegression(penalty='elasticnet', l1_ratio=elastic_net.l1_ratio_, C=1/elastic_net.alpha_, solver='saga', random_state=42)
log_reg.fit(X_train, y_train)

# Making predictions on the test set
y_pred = log_reg.predict(X_test)

##### Step 5: Evaluating the base model

In [ ]:
from sklearn.metrics import recall_score, fbeta_score, precision_score

# Calculate the F-2 score of the model on the test set
f2_score = fbeta_score(y_test, y_pred, beta=2)
print(f"F-2 score of the logistic regression model: {f2_score:.4f}")

# Calculate the accuracy of the model on the test set
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the logistic regression model: {accuracy * 100:.2f}%")

# Also calculate the recall and precision of the model on the test set
recall = recall_score(y_test, y_pred)
print(f"Recall of the logistic regression model: {recall * 100:.2f}%")
precision = precision_score(y_test, y_pred)
print(f"Precision of the logistic regression model: {precision * 100:.2f}%")

With an F-2 score of 0.0284, the model performs very poorly. It is interesting to see that the high accuracy of the model does not at all translate into a high F-2 score, likely because the model is very poor at predicting churners correctly.

In [ ]:
# Show some examples of correct predictions including the actual and predicted churn values
# Create a DataFrame with test data and add columns for actual and predicted values plus their percentage certainties
correct_pred_df = X_test.copy()

correct_pred_df['Actual_Churn'] = y_test
correct_pred_df['Predicted_Churn'] = y_pred

# Add probability predictions (certainty percentages) using predict_proba which returns the probability of each class
# In the case of a 2-class problem, it will use the sigmoid function to calculate the probability of the positive class
y_pred_proba = log_reg.predict_proba(X_test)
correct_pred_df['Probability_Positive_Class'] = [prob[1] * 100 for prob in y_pred_proba]

# Filter for correct predictions
correct_pred_df = correct_pred_df[correct_pred_df['Actual_Churn'] == correct_pred_df['Predicted_Churn']]

# Print nr of correct predictions
print(f"Number of correct predictions: {correct_pred_df.shape[0]}")
print(f"Percentage of correct predictions: {correct_pred_df.shape[0] / len(y_test) * 100:.2f}%")

# Display a few examples of correct predictions
print("\nExamples of correct predictions:")
correct_pred_df.head(10)

In [ ]:
# Show some examples of incorrect predictions including the actual and predicted churn values
# Create a DataFrame with test data and add columns for actual and predicted values
incorrect_pred_df = X_test.copy()
incorrect_pred_df['Actual_Churn'] = y_test
incorrect_pred_df['Predicted_Churn'] = y_pred

# Add probability predictions (certainty percentages) using predict_proba which returns the probability of each class
# In the case of a 2-class problem, it will use the sigmoid function to calculate the probability of the positive class
y_pred_proba = log_reg.predict_proba(X_test)
incorrect_pred_df['Probability_Positive_Class'] = [prob[1] * 100 for prob in y_pred_proba]

# Filter for incorrect predictions
incorrect_pred_df = incorrect_pred_df[incorrect_pred_df['Actual_Churn'] != incorrect_pred_df['Predicted_Churn']]
# Print nr of incorrect predictions
print(f"Number of incorrect predictions: {incorrect_pred_df.shape[0]}")
print(f"Percentage of incorrect predictions: {incorrect_pred_df.shape[0] / len(y_test) * 100:.2f}%")

# Display a few examples of incorrect predictions
print("\nExamples of incorrect predictions:")
incorrect_pred_df.head(10)

In [ ]:
# Display confusion matrix for the logistic regression model
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Create a DataFrame to display the confusion matrix
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Actual Not Churned', 'Actual Churned'], columns=['Predicted Not Churned', 'Predicted Churned'])

# Display the confusion matrix
conf_matrix_df

Problem with the model: it predicts almost all customers not to churn and therefore there is no good indication it can even predict a customer to churn. This makes this model pretty useless for our business case. 

We can find a better decision threshold by examining the ROC curve to balance recall and false positive rate, then choosing a threshold that produces a better F-2 score

##### Step 6: Choosing the right decision threshold

In [ ]:
# Create ROC curve for the logistic regression model
from sklearn.metrics import roc_curve, roc_auc_score

# Calculate the FPR, TPR, and thresholds for the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba[:, 1])

# Calculate the AUC score
auc_score = roc_auc_score(y_test, y_pred_proba[:, 1])

# Plot the ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', label=f'Logistic Regression (AUC = {auc_score:.3f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Guessing')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Logistic Regression')
plt.grid(True, alpha=0.3)
plt.legend(loc='lower right')
plt.show()

# Print additional information about the ROC curve
print(f"AUC Score: {auc_score:.3f}")
# Print the number of threshold points (equal to the number of unique probabilities of probability outputs of the model)
print(f"Number of possible threshold points: {len(thresholds)}")

Considering that an AUC score of 0.5 is equivalent to random guessing in a classification task, the AUC of 0.661 is quite poor

In [ ]:
# Extract TP, TN, FP, and FN from the confusion matrix
_thresholdTP = conf_matrix_df.loc['Actual Churned', 'Predicted Churned']
TN = conf_matrix_df.loc['Actual Not Churned', 'Predicted Not Churned']
FP = conf_matrix_df.loc['Actual Not Churned', 'Predicted Churned']
FN = conf_matrix_df.loc['Actual Churned', 'Predicted Not Churned']

# Create a new dataframe to store threshold and F-2 score
f2_scores = []

for threshold in thresholds:
    # Predict using the current threshold
    y_pred_threshold = [1 if prob[1] >= threshold else 0 for prob in y_pred_proba]
    
    # Calculate TP, TN, FP, FN for this threshold
    TP_threshold = sum((a == 1) and (p == 1) for a, p in zip(y_test, y_pred_threshold))
    TN_threshold = sum((a == 0) and (p == 0) for a, p in zip(y_test, y_pred_threshold))
    FP_threshold = sum((a == 0) and (p == 1) for a, p in zip(y_test, y_pred_threshold))
    FN_threshold = sum((a == 1) and (p == 0) for a, p in zip(y_test, y_pred_threshold))
    
    # Calculate precision and recall
    precisions = TP_threshold / (TP_threshold + FP_threshold) if (TP_threshold + FP_threshold) > 0 else 0
    recalls = TP_threshold / (TP_threshold + FN_threshold) if (TP_threshold + FN_threshold) > 0 else 0

    # Calculate accuracy
    accuracies = (TP_threshold + TN_threshold) / (TP_threshold + TN_threshold + FP_threshold + FN_threshold)
    
    # Calculate F-2 score: (5 * precision * recall) / (4 * precision + recall)
    f2 = (5 * precisions * recalls) / ((4 * precisions) + recalls) if ((4 * precisions) + recalls) > 0 else 0
    f2_scores.append(f2)

f2_df = pd.DataFrame({
    'Threshold': thresholds,
    'F2_Score': f2_scores,
    'Precision': precisions,
    'Recall': recalls,
    'Accuracy': accuracies
})

top_5_thresholds = f2_df.nlargest(5, 'F2_Score')
# Display the top 5 thresholds with highest F-2 score
top_5_thresholds

In [ ]:
# Print out the best threshold (highest accuracy)
best_threshold = top_5_thresholds.iloc[0]['Threshold']
best_f2 = top_5_thresholds.iloc[0]['F2_Score']
print(f"The best threshold is {best_threshold} with an F-2 score of {best_f2:.4f}")

##### Step 7: Evaluate final Logistic Regression model

In [ ]:
# Show some examples of correct predictions including the actual and predicted churn values with a threshold of 0.0896728999861171 
# Create a DataFrame with test data and add columns for actual and predicted values plus their percentage certainties
correct_pred_df_final = X_test.copy()

correct_pred_df_final['Actual_Churn'] = y_test

# Add probability predictions (certainty percentages) using predict_proba which returns the probability of each class
# In the case of a 2-class problem, it will use the sigmoid function to calculate the probability of the positive class
y_pred_proba = log_reg.predict_proba(X_test)
correct_pred_df_final['Probability_Positive_Class'] = [prob[1] * 100 for prob in y_pred_proba]

# Set the threshold for the certainty percentage to 0.0896728999861171 
threshold = best_threshold

# Add column to show predicted churn based on the threshold
correct_pred_df_final['Predicted_Churn'] = [1 if prob/100 >= threshold else 0 for prob in correct_pred_df_final['Probability_Positive_Class']]

# Filter for correct predictions
correct_pred_df_final = correct_pred_df_final[correct_pred_df_final['Actual_Churn'] == correct_pred_df_final['Predicted_Churn']]


# Print nr of correct predictions
print(f"Number of correct predictions with threshold of {threshold}: {correct_pred_df_final.shape[0]}")
print(f"Percentage of correct predictions with threshold of {threshold}: {correct_pred_df_final.shape[0] / len(y_test) * 100:.2f}%")

# Display a few examples of correct predictions
print(f"\nExamples of correct predictions with threshold of {best_threshold}:")
correct_pred_df_final.head(10)

In [ ]:
# Show some examples of incorrect predictions including the actual and predicted churn values with a threshold of 0.135914
# Create a DataFrame with test data and add columns for actual and predicted values
incorrect_pred_df_final = X_test.copy()

incorrect_pred_df_final['Actual_Churn'] = y_test

# Follow steps above
incorrect_pred_df_final['Probability_Positive_Class'] = [prob[1] * 100 for prob in y_pred_proba]

# Add column to show predicted churn based on the threshold
incorrect_pred_df_final['Predicted_Churn'] = [1 if prob/100 >= threshold else 0 for prob in incorrect_pred_df_final['Probability_Positive_Class']]

# Filter for incorrect predictions
incorrect_pred_df_final = incorrect_pred_df_final[incorrect_pred_df_final['Actual_Churn'] != incorrect_pred_df_final['Predicted_Churn']]

# Print nr of incorrect predictions
print(f"Number of incorrect predictions with threshold of {threshold}: {incorrect_pred_df_final.shape[0]}")
print(f"Percentage of incorrect predictions with threshold of {threshold}: {incorrect_pred_df_final.shape[0] / len(y_test) * 100:.2f}%")

# Display a few examples of incorrect predictions
print("\nExamples of incorrect predictions with threshold of 0.135914:")
incorrect_pred_df_final.head(10)

In [ ]:
# Fuse together the correct and incorrect predictions DataFrames with the threshold at 0.135914 to display the confusion matrix
final_pred_df = pd.concat([correct_pred_df_final, incorrect_pred_df_final])

# Print shape of final_pred_df
print(f"Shape of final_pred_df: {final_pred_df.shape}")
final_pred_df.head()

In [ ]:
# Create and display confusion matrix of the final predictions
final_conf_matrix = confusion_matrix(final_pred_df['Actual_Churn'], final_pred_df['Predicted_Churn'])

# Create a DataFrame to display the confusion matrix
final_conf_matrix_df = pd.DataFrame(final_conf_matrix, index=['Actual Not Churned', 'Actual Churned'], columns=['Predicted Not Churned', 'Predicted Churned'])

# Display the confusion matrix
final_conf_matrix_df

In [ ]:
from sklearn.metrics import precision_score, recall_score

# Show confusion matrix of base model and final model next to each other
plt.figure(figsize=(16, 6))
conf_matrix_df
# Create a figure with two subplots side by side
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot confusion matrix of base model (left)
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues', ax=axs[0])
axs[0].set_title('Base Model Confusion Matrix')
axs[0].set_ylabel('Actual')
axs[0].set_xlabel('Predicted')

# Plot confusion matrix of final model (right)
sns.heatmap(final_conf_matrix_df, annot=True, fmt='d', cmap='Blues', ax=axs[1])
axs[1].set_title('Final Model with Optimized Threshold')
axs[1].set_ylabel('Actual')
axs[1].set_xlabel('Predicted')

plt.tight_layout()
plt.show()

# Calculate F-2 score, precision, recall and accuracy of the final model
final_f2_score = fbeta_score(final_pred_df['Actual_Churn'], final_pred_df['Predicted_Churn'], beta=2)
final_precision = precision_score(final_pred_df['Actual_Churn'], final_pred_df['Predicted_Churn'])
final_recall = recall_score(final_pred_df['Actual_Churn'], final_pred_df['Predicted_Churn'])
final_accuracy = accuracy_score(final_pred_df['Actual_Churn'], final_pred_df['Predicted_Churn'])

# Print the F-2 score, precision, recall, and accuracy of the base model and final model
print(f"Base Model - F-2 Score: {f2_score:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}")
print(f"Final Model - F-2 Score: {final_f2_score:.4f}, Precision: {final_precision:.4f}, Recall: {final_recall:.4f}, Accuracy: {final_accuracy:.4f}")

The final model has a bit higher accuracy, however, it is even worse at predicting churn correctly. Therefore, we will now use a random forest with XGBoost in the hopes of creating a better model

#### Model 2: XGBoost

This model is chosen because while it is not as interpretable as a logistic regression, XGBoost is highly relevant for customer churn prediction because of its ability to handle structured data, detect complex nonlinear patterns, and deal with imbalanced datasets. Additionally, it has regularization capabilities and the importance of each feature can be determined to get an insight into why the model performs the way it does.

##### Step 1: Feature Selection

In [ ]:
# For this model, we leave in all features, including the state, to give the model the best possible chance of predicting churn well.
# Transform state column into dummy variables
model2_df = pd.get_dummies(df, columns=['State'], drop_first=True)

# Create binary variables for categorical columns (International plan, voice mail plan, and state)
model2_df['International plan'] = model2_df['International plan'].map({'Yes': 1, 'No': 0})
model2_df['Voice mail plan'] = model2_df['Voice mail plan'].map({'Yes': 1, 'No': 0})
model2_df.head()

In [ ]:
# We will also not drop the columns that are perfectly linearly correlated because a random forest can better handle multicollinearity

In [ ]:
# Also, again change Churn column from boolean to int with Churn = False as 0 and Churn = True as 1
model2_df['Churn'] = model2_df['Churn'].astype(int)
model2_df.head()

In [ ]:
# Finally, for this model we will add additional features
# Add total charge column
model2_df['Total charge'] = model2_df['Total day charge'] + model2_df['Total eve charge'] + model2_df['Total night charge'] + model2_df['Total intl charge']

# Add total minutes column
model2_df['Total minutes'] = model2_df['Total day minutes'] + model2_df['Total eve minutes'] + model2_df['Total night minutes'] + model2_df['Total intl minutes']

# Add total calls column
model2_df['Total calls'] = model2_df['Total day calls'] + model2_df['Total eve calls'] + model2_df['Total night calls'] + model2_df['Total intl calls']

# Add average call duration
model2_df['Average call duration'] = model2_df['Total minutes'] / model2_df['Total calls']

# Add Average Charge per Call
model2_df['Average charge per call'] = model2_df['Total charge'] / model2_df['Total calls']
model2_df.head()

##### Step 2: Train-test split

In [ ]:
# Use same train-test split as before
X_2 = model2_df.drop(['Churn'], axis=1)  # Features
y_2 = model2_df['Churn']  # Target variable

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=41, stratify=y_2)

# Check the shapes of the train and test sets
print(f"X_train_2 shape: {X_train_2.shape}")
print(f"X_test_2 shape: {X_test_2.shape}")
print(f"y_train_2 shape: {y_train_2.shape}")
print(f"y_test_2 shape: {y_test_2.shape}")

# Check the distribution of the target variable in train and test sets to ensure that stratify=y worked
print(f"Churn rate in training set: {y_train.mean() * 100:.2f}%")
print(f"Churn rate in test set: {y_test.mean() * 100:.2f}%")

##### Step 3: Hyperparameter tuning

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier

# Create a pipeline that applies SMOTE bc of class imbalance and then trains a decision tree classifier
# The pipeline will be used to fit the model to the training data
pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('xgb', XGBClassifier(random_state=42))
])

In [ ]:
# Set up the hyperparameter grid for the decision tree classifier

param_grid = {
    'xgb__base_score': [0.5],         # Initial prediction score for all instances, lower values predict less churn
    'xgb__booster': ['gbtree'],  # Algorithm to use - tree-based, linear model
    'xgb__colsample_bylevel': [1.0],        # Subsample ratio of columns for each level within a tree
    'xgb__colsample_bytree': [1.0],         # Subsample ratio of columns when constructing each tree
    'xgb__gamma': [0, 0.1],            # Minimum loss reduction required for further partition on a leaf node
    'xgb__n_estimators': [75, 100],   # Number of trees to build (more trees = better performance but slower)
    'xgb__max_depth': [5, 10],    # Maximum depth of a tree (controls complexity)
    'xgb__learning_rate': [0.9, 0.1],   # Step size shrinkage to prevent overfitting
    'xgb__min_child_weight': [1, 2],        # Minimum sum of instance weight needed in a child
    'xgb__objective': ['binary:logistic'],   # Defines the loss function (binary classification with logistic)
    'xgb__random_state': [42],              # Random number seed for reproducibility
    'xgb__reg_alpha': [0, 0.1],        # L1 regularization on weights (prevents overfitting)
    'xgb__reg_lambda': [1, 2],           # L2 regularization on weights (prevents overfitting)
    'xgb__subsample': [1]                   # Subsample ratio of training instances (1 = use all samples)
}

In [ ]:
from sklearn.model_selection import GridSearchCV

# Set up stratified 5-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Set up the grid search with the pipeline, parameter grid, and cross-validation
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=cv,
    scoring='accuracy',  # Use accuracy as the scoring metric
    n_jobs=1,  # Use one CPU core
    verbose=2,  # Provide detailed output
)

# Fit the grid search to the training data
grid_search.fit(X_train_2, y_train_2)

In [ ]:
# Output best parameters and cross-validation accuracy
print("Best Hyperparameters:", grid_search.best_params_)
print("Best CV Accuracy: {:.2f}%".format(grid_search.best_score_*100))

##### Step 4: Evaluating the Model

In [ ]:
# Evaluate on the test set using the best found model
best_model = grid_search.best_estimator_
y_pred_2 = best_model.predict(X_test_2)

# Calculate the accuracy of the model on the test set
accuracy_2 = accuracy_score(y_test_2, y_pred_2)
print(f"Accuracy of the XGBoost model: {accuracy_2 * 100:.2f}%")

# Also calculate the recall of the model on the test set because when predicting churn, we want to minimize false negatives
recall_2 = recall_score(y_test_2, y_pred_2)
print(f"Recall of the XGBoost model: {recall_2 * 100:.2f}%")

In [ ]:
# Show some examples of correct predictions including the actual and predicted churn values
# Create a DataFrame with test data and add columns for actual and predicted values plus their percentage certainties
correct_pred_df_2 = X_test_2.copy()

correct_pred_df_2['Actual_Churn'] = y_test_2
correct_pred_df_2['Predicted_Churn'] = y_pred_2

# Filter for correct predictions
correct_pred_df_2 = correct_pred_df_2[correct_pred_df_2['Actual_Churn'] == correct_pred_df_2['Predicted_Churn']]

# Print nr of correct predictions
print(f"Number of correct predictions: {correct_pred_df_2.shape[0]}")
print(f"Percentage of correct predictions: {correct_pred_df_2.shape[0] / len(y_test_2) * 100:.2f}%")

# Display a few examples of correct predictions
print("\nExamples of correct predictions:")
correct_pred_df_2.tail(10)

In [ ]:
# Show some examples of incorrect predictions including the actual and predicted churn values
# Create a DataFrame with test data and add columns for actual and predicted values
incorrect_pred_df_2 = X_test_2.copy()
incorrect_pred_df_2['Actual_Churn'] = y_test_2
incorrect_pred_df_2['Predicted_Churn'] = y_pred_2

# Filter for incorrect predictions
incorrect_pred_df_2 = incorrect_pred_df_2[incorrect_pred_df_2['Actual_Churn'] != incorrect_pred_df_2['Predicted_Churn']]
# Print nr of incorrect predictions
print(f"Number of incorrect predictions: {incorrect_pred_df_2.shape[0]}")
print(f"Percentage of incorrect predictions: {incorrect_pred_df_2.shape[0] / len(y_test_2) * 100:.2f}%")

# Display a few examples of incorrect predictions
print("\nExamples of incorrect predictions:")
incorrect_pred_df_2.head(18)

In [ ]:
# Display confusion matrix for the XGBoost Random Forest Model
# Calculate the confusion matrix
conf_matrix_2 = confusion_matrix(y_test_2, y_pred_2)

# Create a DataFrame to display the confusion matrix
conf_matrix_df_2 = pd.DataFrame(conf_matrix_2, index=['Actual Not Churned', 'Actual Churned'], columns=['Predicted Not Churned', 'Predicted Churned'])

# Display the confusion matrix
conf_matrix_df_2

##### Step 5: Comparing both models

In [ ]:
# Comparing the confusion matrices of the logistic regression model and the XGBoost Random Forest model
plt.figure(figsize=(16, 6))
conf_matrix_df
# Create a figure with two subplots side by side
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot confusion matrix of logistic regression model (left)
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues', ax=axs[0])
axs[0].set_title('Logistic Regression Model Confusion Matrix')
axs[0].set_ylabel('Actual')
axs[0].set_xlabel('Predicted')

# Plot confusion matrix of XGBoost Random Forest model (right)
sns.heatmap(conf_matrix_df_2, annot=True, fmt='d', cmap='Blues', ax=axs[1])
axs[1].set_title('XGBoost Random Forest Model Confusion Matrix')
axs[1].set_ylabel('Actual')
axs[1].set_xlabel('Predicted')

plt.tight_layout()
plt.show()

# Print precision, recall and accuracy of both models
print(f"Logistic Regression Model - Recall: {recall:.2f}, Accuracy: {accuracy:.2f}")
print(f"XGBoost Random Forest Model - Recall: {recall_2:.2f}, Accuracy: {accuracy_2:.2f}")

In [ ]:
# Comparing the ROC curves of the logistic regression model and the XGBoost Random Forest model
# Calculate the FPR, TPR, and thresholds for the ROC curve
fpr_2, tpr_2, thresholds_2 = roc_curve(y_test_2, best_model.predict_proba(X_test_2)[:, 1])

# Calculate the AUC score
auc_score_2 = roc_auc_score(y_test_2, best_model.predict_proba(X_test_2)[:, 1])

# Plot the ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', label=f'Logistic Regression (AUC = {auc_score:.3f})')
plt.plot(fpr_2, tpr_2, color='green', label=f'XGBoost Random Forest (AUC = {auc_score_2:.3f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Guessing')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Logistic Regression vs XGBoost Random Forest')
plt.grid(True, alpha=0.3)
plt.legend(loc='lower right')
plt.show()

The XGBoost model is much better at predicting churned customers than the logistic regression and only marginally worse at predicting non-churners. Therefore, it makes sense to use this model going forward

##### Step 6: Feature Importance

In [ ]:
# Looking at plot of feature importance
import xgboost as xgb

# Access the XGBClassifier from the pipeline
xgb_model_classifier = best_model.named_steps['xgb']

# Plot feature importances using xgboost's built-in function
fig, ax = plt.subplots(figsize=(10, 10))
# Gain is used to measure the improvement in accuracy brought by a feature to the branches it is on because 
# accuracy is the main goal of the model
xgb.plot_importance(xgb_model_classifier, importance_type="gain", ax=ax, max_num_features=20)
plt.title('Feature Importance from XGBoost Model')
plt.show()

### Q3.1 Which performance metric did you use to evaluate the performance? Why? [Free text + code]

I selected accuracy as the performance metric because that is the metric the professor said the models will have their performance graded on. Otherwise, I would have chosen the F-2 score. The F-2 score merges recall and precision into a single score, providing a good blend between how well the model can predict churners well and if it overpredicts non-churners to be churners. Unlike the F-1 score, the F-2 score puts more emphasis on recall than on precision. This makes sense because the telcom company should be more concerned about correctly predicting all possible churning customers (recall) than predicting too many non-churning customers to be churners (precision) because a churned customer likely costs the company a lot more than being overcareful with a non-churning customer. On the other hand, just focusing entirely on recall will incentivize the model just to predict all customers as churners which is not ideal because it will involve a lot of unnecessary effort on the part of the telcom company to keep customers happy that are not planning on leaving in the first place. Therefore, the F-2 score provides a very good balance and makes most sense to me as a performance metric.

### Q3.2 Which model provided the best results?

As shown previously, the XGBoost model provided the best results both in terms of accuracy and recall when compared to the logistic regression. The ROC curves of both models show that this pattern holds at almost all threshold levels.

## Q4: The executive team has provided an external validation dataset with the same structure as the original data. To test the best model, the team must ensure it works seamlessly on new data. The model should predict churn using binary outputs: 0 (No Churn) and 1 (Churn).

- Save the best-performing model as a Pickle file (StudentNumber_Pipeline.pkl).
Please see the code Assignment1_SaveAsPickle.ipynb with the examples
- To ensure the code runs in every machine please save the requirements files with the
name StudentNumber_requirements.txt – please see
Assignment1_SaveAsPickle.ipynb on how to do it.
    - For this to happen you will also require submitting a .py file with your feature
engineer function, please keep the following name feature_engineering.py
- If you are using a model outside the scikit-learn you will need to save the following
files:
    - StudentNumber_Preprocessor.pkl – where the preprocessing steps are
performed
    - StudentNumber_Model.pkl - where the model does the prediction task.
- Write code to reload the saved model and test it on the external validation dataset
(please see Assignment1_SaveAsPickle.ipynb on how to do it. The validation set
will be in the format from the file
2767ML_assignment1_externalvalidation_data_toStudents.csv). If your
model runs in this dataset it will run in the final validation set.
- Ensure the model outputs predictions in the required format 0 (No Churn) and 1
(Churn).

How This Question Will Be Evaluated:
1. Top 25% Models: The models that perform the best on the external validation dataset
will receive full marks for this task. Performance will be ranked based on accuracy.
2. Next 25% Models: Students whose models perform in the following 25% will have 3
and so on.
3. Non-Functional Models: If your model fails to load, run, or provide predictions in the
required format (0 for No Churn, 1 for Churn), you will receive zero marks for this
task. Make sure your .pkl file and code are functional, tested, and well-documented.
    1. You can find a test set in the assignment on moodle with the same format
as the final (2767ML_assignment1_externalvalidation_data_toStudents.csv).

### Step 1: Saving the model

In [ ]:
# Load initial dataset
df = pd.read_csv('2767ML_assignment1_data.csv')

# Identify numerical and categorical columns
df.dtypes

In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import pickle

# Save numerical columns to a list
numerical_columns = ['Account length', 'Area code', 'Number vmail messages', 'Total day minutes', 'Total day calls', 'Total day charge', 'Total eve minutes', 'Total eve calls', 'Total eve charge', 'Total night minutes', 'Total night calls', 'Total night charge', 'Total intl minutes', 'Total intl calls', 'Total intl charge', 'Customer service calls']

# Save categorical columns to a list
categorical_columns = ['State', 'International plan', 'Voice mail plan']

# Convert Churn from boolean to integer (0 for False, 1 for True)
df['Churn'] = df['Churn'].astype(int)

# Split the data into features and target variable
X_save = df.drop('Churn', axis=1)
Y_save = df['Churn']

# Define function to perform feature selection steps
def feature_engineering(X_save):
    # Add total charge column
    X_save['Total charge'] = X_save['Total day charge'] + X_save['Total eve charge'] + X_save['Total night charge'] + X_save['Total intl charge']
    # Add total minutes column
    X_save['Total minutes'] = X_save['Total day minutes'] + X_save['Total eve minutes'] + X_save['Total night minutes'] + X_save['Total intl minutes']
    # Add total calls column
    X_save['Total calls'] = X_save['Total day calls'] + X_save['Total eve calls'] + X_save['Total night calls'] + X_save['Total intl calls']
    # Add average call duration
    X_save['Average call duration'] = X_save['Total minutes'] / X_save['Total calls']
    # Add Average Charge per Call
    X_save['Average charge per call'] = X_save['Total charge'] / X_save['Total calls']

    return X_save

# Apply FunctionTransformer to apply the feature engineering function
feature_transformer = FunctionTransformer(feature_engineering)

# Handle NaN values in the dataset
numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # Replace NaN with median
    ('scaler', StandardScaler())  # Scale the numerical data
])

# Handle categorical values in the dataset
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=np.nan)),  # Replace NaN with 'missing'
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode the categorical data
])

# 3 Combine both transformations into a ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', numerical_transformer, numerical_columns),
    ('cat', categorical_transformer, categorical_columns)
])

# Create the full pipeline with preprocessing + feature engineering + model
preprocessing_pipeline = Pipeline([
    ('feature_engineering', feature_transformer), 
    ('preprocessor', preprocessor),
])

# Apply preprocessing transformations
X_transformed = preprocessing_pipeline.fit_transform(X_save)

# Train-test split
X_train_save, X_test_save, Y_train_save, Y_test_save = train_test_split(X_transformed, Y_save, test_size=0.20, random_state=42)

# Train **XGBoost** Model
xgb_model = best_model
xgb_model.fit(X_train_save, Y_train_save)

# Save **Preprocessing + Feature Engineering Together**
with open("preprocessor.pkl", "wb") as file:
    pickle.dump(preprocessing_pipeline, file)

# Save **XGBoost Model** as Pickle
with open("Finn_Hetzler_Model.pkl", "wb") as file:
    pickle.dump(xgb_model, file)  # Saving XGBClassifier using Pickle

print("Preprocessing pipeline (feature engineering + transformations) and XGBoost model saved successfully!")

### Step 2: Install the requirements, load the model and apply it

In [ ]:
!pip freeze > requirements.txt

In [ ]:
from feature_engineering import feature_engineering
import pickle
import pandas as pd

# Load the preprocessing pipeline (including feature engineering)
with open("preprocessor.pkl", "rb") as file:
    loaded_preprocessor = pickle.load(file)

# Load the XGBoost model (pickled)
with open("Finn_Hetzler_Model.pkl", "rb") as file:
    xgb_loaded_model = pickle.load(file)

print("Preprocessing pipeline (feature engineering + transformations) and XGBoost model loaded successfully!")

# New data to make a prediction
validation_df = pd.read_csv('2767ML_assignment1_externalvalidation_data_toStudents.csv')
validation_df.head()

# Apply feature engineering and preprocessing together
new_data_transformed = loaded_preprocessor.transform(validation_df)

# Make prediction using the loaded XGBoost model
xgb_prediction = xgb_loaded_model.predict(new_data_transformed)

print(f"XGBoost Prediction: {xgb_prediction[0]}")

## Q5: The executive team requires actionable insights to guide the strategy to address customer churn effectively. Your analysis will directly inform their decisions

### Q5.1 What customer characteristics most strongly influence churn? [Free text]

The most important features were already shown earlier in the plot in Step 6 of Model 2: [Link](#step-6-feature-importance). Note, I wanted to put the plot down here but for some reason the feature names were not showing

Based on this plot, the most important features are the Total Charge, Customer service calls and Number vmail messages of the customer. It is particularly noteworthy that Total Charge was the most important feature, considering all of the features that were summed up to make this feature (Total day charge, Total eve charge, Total night charge, Total intl charge) were left in the model and therefore likely take away feature importance from Total Charge.

### Q5.2 What actionable steps should the company take to reduce churn? Suggest two strategies. [Free text]

1. The company should collect feedback from customers the model predicts are likely to churn. By proactively asking these people, for example with a survey, they can address the issues they have with the current service, so the company knows what to change so that they remain customers.

2. As seen above in the 15 most important features, one of the states, namely "WV" or West Virginia is also an important feature. It could also make sense to take a closer look at what customers are unsatisfied with there. For example, if they say network quality is not good, then they can look into improving their network infrastructure. Alternatively, if there is a cheaper competitor, they can look into lowering prices to remain competitive.